In [1]:
from dotenv import load_dotenv
import os

load_dotenv()


LANGUAGE_KEY = os.getenv('LANGUAGE_KEY')
LANGUAGE_ENDPOINT = os.getenv('LANGUAGE_ENDPOINT')

In [2]:
import docx 

# doc =  docx.Document("data/2024_12_06_Réunion d'équipe.docx")
doc =  docx.Document("data/2024-12-208Réunion-d'équipe.docx")

In [ ]:
for i in range(2,10):
    print(doc.paragraphs[i].text)

Predefined Roles:

"Customer":
"Agent":

In [ ]:
people = ("list of names from the transcript file")
company = dict({"name1":{'id':1,
                                    'role':'Customer'},
                  "name2":{'id':2,
                                   'role':'Customer'},
                  "name3":{'id':3,
                                   'role':'Assistant'},
                  "name4":{'id':4,
                                    'role':'Customer'},                    
                  })



In [ ]:

def assign_role_and_id(line):
    nom = " ".join(line.split(" ")[:2])
    participant_id = company[nom]["id"]
    role = company[nom]["role"]
    return participant_id, role

formatted_paragraphs=list()
characters_count=0
for i, paragraph in enumerate(doc.paragraphs):
    text=list()
    if i <= 2:
        continue
    # Format each paragraph as a dictionary
    for line in paragraph.text.split("\n"):
        
        if line == "":
            continue
        
        elif line.startswith(people):  
            participant_id, role = assign_role_and_id(line)
            

        else:
            if line:
                text.append(line.replace("\xa0",""))
                
                characters_count += len(line.replace("\xa0",""))
            
    formatted_paragraph = {
        "text": ". ".join(text),
        "id": str(i),  # Assign a unique ID based on the order
        "role": role,      # Assign a role (adjust based on your document content)
        "participantId": participant_id
    }
    
    # Append formatted paragraph to list
    formatted_paragraphs.append(formatted_paragraph)
    
print(characters_count,"characters")
print(characters_count/4,"tokens")

38419 characters
9604.75 tokens


In [8]:
text = " ".join([formatted_paragraphs[i]['text'] for i in range(len(formatted_paragraphs))])

In [9]:
import unicodedata
sum(1 for _ in unicodedata.normalize('NFC', text))

39372

In [20]:
import unicodedata

def get_text_length(text):
    """Calculate the length of a string in Unicode text elements."""
    return sum(1 for _ in unicodedata.normalize('NFC', text))

def chunk_text_by_length(conversation, max_length):
    """Split the conversation into chunks based on text length in Unicode elements."""
    chunks = []
    current_chunk = []
    current_length = 0

    for item in conversation:
        text_length = get_text_length(item['text'])
        if current_length + text_length <= max_length:
            current_chunk.append(item)
            current_length += text_length
        else:
            chunks.append(current_chunk)
            current_chunk = [item]
            current_length = text_length

    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

chunks = chunk_text_by_length(formatted_paragraphs,max_length=12500)

print(len(chunks))

3


In [ ]:
chunks

In [18]:
def view_result(result):
    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']}: {summary['text']}")

In [ ]:
import unicodedata
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient


# conversation_chunks = chunk_text_by_length(formatted_paragraphs,max_length=4500)

conversation_chunks = chunks[1:]

client = ConversationAnalysisClient(LANGUAGE_ENDPOINT, AzureKeyCredential(LANGUAGE_KEY))

try:
    results = []
    for idx, chunk in enumerate(conversation_chunks):
        conversation_id = f"meeting_group{idx + 1}"
        poller = client.begin_conversation_analysis(
            task={
                "displayName": f"Analyze conversations part {idx + 1}",
                "analysisInput": {
                    "conversations": [
                        {
                            "conversationItems": chunk,
                            "modality": "text",
                            "id": conversation_id,
                            "language": "fr",
                        },
                    ]
                },
                "tasks": [
                    # {
                    #     "taskName": "Issue task",
                    #     "kind": "ConversationalSummarizationTask",
                    #     "parameters": {"summaryAspects": ["issue"]},
                    # },
                    # {
                    #     "taskName": "Resolution task",
                    #     "kind": "ConversationalSummarizationTask",
                    #     "parameters": {"summaryAspects": ["resolution"]},
                    # },
         
                    {
                        "taskName": "Title and chapter task",
                        "kind": "ConversationalSummarizationTask",
                        "parameters": {"summaryAspects": ["chapterTitle"]},
                    },
                    {
                        "taskName": "Recap task narrative",
                        "kind": "ConversationalSummarizationTask",
                        "parameters": {"summaryAspects": ["narrative"]},
                    },
                    {
                        "taskName": "Recap task",
                        "kind": "ConversationalSummarizationTask",
                        "parameters": {"summaryAspects": ["recap"]},
                    },
                ],
            }
        )
        result = poller.result()
        results.append(result)
        view_result(result)
        
except Exception as e:
    print(e)
    
finally:
    client.close()


# create doc document

In [28]:
from docx import Document


recap = Document()

recap.add_heading('Resume equipe 2024-12-20')
recap.add_paragraph("Ce resume utilise l'outil Microsoft Azure ConversationAnalysisClient avec les paramètres 'summaryAspects': [chapterTitle] & [narrative]")
for i, result in enumerate(results):

    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status {i+1}: {task['status']}")
        recap.add_heading(f"\n{task['taskName']} status {i}: {task['status']}", level=1)
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']} {i+1}: {summary['text']}")
                    recap.add_paragraph(f"{summary['aspect']} {i}: {summary['text']}")
                    
                    
recap.save("2024-12-20_recap_reunion_equipe.docx")


Feature	Value

Text Analytics for health	125,000 characters as measured by StringInfo.LengthInTextElements.

All other preconfigured features (synchronous)	5,120 as measured by StringInfo.LengthInTextElements. If you need to submit larger documents, consider using the feature asynchronously.

All other preconfigured features (asynchronous)	125,000 characters across all submitted documents, as measured by StringInfo.LengthInTextElements (maximum of 25 documents).

In [ ]:
chunks = chunk_text_by_length(formatted_paragraphs,max_length=125000)

print(len(chunks))

1


In [ ]:
# from azure.ai.language.conversations.aio import ConversationAnalysisClient
# import asyncio

# async def analyze_conversations_async(language_endpoint, language_key, conversation_chunk):
#     client = ConversationAnalysisClient(language_endpoint, AzureKeyCredential(language_key))

#     try:
#         conversation_id = f"meeting_summary_group_meeting_{ 1}"
        
#         # Use `await` for asynchronous operations
#         poller = await client.begin_conversation_analysis(
#             task={
#                 "displayName": f"Analyze conversations part {1}",
#                 "analysisInput": {
#                     "conversations": [
#                         {
#                             "conversationItems": conversation_chunk,
#                             "modality": "text",
#                             "id": conversation_id,
#                             "language": "fr",
#                         },
#                     ]
#                 },
#                 "tasks": [
#                     {
#                         "taskName": "Title and chapter task",
#                         "kind": "ConversationalSummarizationTask",
#                         "parameters": {"summaryAspects": ["chapterTitle"]},
#                     },
#                     {
#                         "taskName": "Recap task",
#                         "kind": "ConversationalSummarizationTask",
#                         "parameters": {"summaryAspects": ["recap"]},
#                     },
#                 ],
#             }
#         )
#         # Await the result of the poller
#         result = await poller.result()
#         view_result(result)

#     finally:
#         # Use `await` for closing the client
#         await client.close()

#     return results


# results = await analyze_conversations_async(LANGUAGE_ENDPOINT, LANGUAGE_KEY, formatted_paragraphs)


In [ ]:
poller = text_analytics_client.begin_analyze_actions(
    documents,
    display_name="Sample Text Analysis",
    actions=[
        RecognizeEntitiesAction(),
        RecognizePiiEntitiesAction(),
        ExtractKeyPhrasesAction(),
        RecognizeLinkedEntitiesAction(),
        AnalyzeSentimentAction(),
    ],
)


document_results = poller.result()
for doc, action_results in zip(documents, document_results):
    print(f"\nDocument text: {doc}")
    for result in action_results:
        if result.kind == "EntityRecognition":
            print("...Results of Recognize Entities Action:")
            for entity in result.entities:
                print(f"......Entity: {entity.text}")
                print(f".........Category: {entity.category}")
                print(f".........Confidence Score: {entity.confidence_score}")
                print(f".........Offset: {entity.offset}")

        elif result.kind == "PiiEntityRecognition":
            print("...Results of Recognize PII Entities action:")
            for pii_entity in result.entities:
                print(f"......Entity: {pii_entity.text}")
                print(f".........Category: {pii_entity.category}")
                print(f".........Confidence Score: {pii_entity.confidence_score}")

        elif result.kind == "KeyPhraseExtraction":
            print("...Results of Extract Key Phrases action:")
            print(f"......Key Phrases: {result.key_phrases}")

        elif result.kind == "EntityLinking":
            print("...Results of Recognize Linked Entities action:")
            for linked_entity in result.entities:
                print(f"......Entity name: {linked_entity.name}")
                print(f".........Data source: {linked_entity.data_source}")
                print(f".........Data source language: {linked_entity.language}")
                print(
                    f".........Data source entity ID: {linked_entity.data_source_entity_id}"
                )
                print(f".........Data source URL: {linked_entity.url}")
                print(".........Document matches:")
                for match in linked_entity.matches:
                    print(f"............Match text: {match.text}")
                    print(f"............Confidence Score: {match.confidence_score}")
                    print(f"............Offset: {match.offset}")
                    print(f"............Length: {match.length}")

        elif result.kind == "SentimentAnalysis":
            print("...Results of Analyze Sentiment action:")
            print(f"......Overall sentiment: {result.sentiment}")
            print(
                f"......Scores: positive={result.confidence_scores.positive}; \
                neutral={result.confidence_scores.neutral}; \
                negative={result.confidence_scores.negative} \n"
            )

        elif result.is_error is True:
            print(
                f"...Is an error with code '{result.error.code}' and message '{result.error.message}'"
            )

    print("------------------------------------------")

In [ ]:

        

# # Combine results as needed
# for idx, result in enumerate(results):
#     print(f"Results for part {idx + 1}: {result}")


In [ ]:
all_texts = " ".join([results[i]['tasks']['items'][0]['results']['conversations'][0]['summaries'][0]['text'] for i in range(len(results))])


Extractive summarization: Produces a summary by extracting sentences that collectively represent the most important or relevant information within the original content.

Abstractive summarization: Produces a summary by generating summarized sentences from the document that capture the main idea.

Query-focused summarization: Allows you to use a query when summarizing.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient


text_analytics_client = TextAnalyticsClient(LANGUAGE_ENDPOINT, AzureKeyCredential(LANGUAGE_KEY))


In [ ]:
poller = text_analytics_client.begin_extract_summary([all_texts])
extract_summary_results = poller.result()
for result in extract_summary_results:
    if result.kind == "ExtractiveSummarization":
        print("Summary extracted: \n{}".format(
            " ".join([sentence.text for sentence in result.sentences]))
        )
    elif result.is_error is True:
        print("...Is an error with code '{}' and message '{}'".format(
            result.error.code, result.error.message
        ))
